In [1]:
!pip install pymongo

     ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
     ------------------- ------------------ 245.8/472.7 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 472.7/472.7 kB 4.9 MB/s eta 0:00:00
  Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)


In [6]:
from pymongo import MongoClient

In [8]:
# Membuat koneksi ke server MongoDB

client = MongoClient('localhost', 27017)


In [11]:
# Membuat database baru bernama 'demoDB'

db = client['demoDB']


In [10]:
# Menyisipkan dokumen ke dalam koleksi 'users'

db.users.insert_one({'name': 'John', 'age': 25, 'address': '123 Main St', 'gender': 'male'})


InsertOneResult(ObjectId('654a1989d61c91fe5de86631'), acknowledged=True)

In [13]:
database_names = client.list_database_names()
print(database_names)


['admin', 'config', 'local', 'populations']


In [14]:
# Menampilkan seluruh dokumen dalam koleksi 'users'

for user in db.users.find():
    print(user)

In [16]:
# Mencari dokumen dengan kriteria tertentu

for user in db.users.find({'name': 'John'}):
    print(user)

In [20]:
# Memperbarui dokumen

db.users.update_one({'name': 'John'}, {'$set': {'age': 26}})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [21]:
# Menghapus dokumen

db.users.delete_one({'name': 'John'})


DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [18]:
# Menambahkan beberapa dokumen

db.users.insert_many([
    {'name': 'Alice', 'age': 28, 'address': '456 Elm St', 'gender': 'female'},
    {'name': 'Bob', 'age': 32, 'address': '789 Maple St', 'gender': 'male'},
    {'name': 'Charlie', 'age': 22, 'address': '101 Pine St', 'gender': 'male'},
    {'name': 'Diana', 'age': 29, 'address': '202 Oak St', 'gender': 'female'}
])


InsertManyResult([ObjectId('654a1b25d61c91fe5de86632'), ObjectId('654a1b25d61c91fe5de86633'), ObjectId('654a1b25d61c91fe5de86634'), ObjectId('654a1b25d61c91fe5de86635')], acknowledged=True)

In [19]:
# Mencari dengan query operator

for user in db.users.find({'age': {'$gt': 30}}):
    print(user)

{'_id': ObjectId('654a1b25d61c91fe5de86633'), 'name': 'Bob', 'age': 32, 'address': '789 Maple St', 'gender': 'male'}


In [35]:
# Menghitung jumlah dokumen dengan kriteria tertentu
count = db.users.count_documents({'age': {'$gt': 30}})
print(count)

1


In [22]:
# Membuat index pada koleksi untuk optimalisasi query

db.users.create_index([('name', 1)])


'name_1'

In [23]:
# Menampilkan index yang ada pada koleksi

indexes = db.users.index_information()
print(indexes)

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'name_1': {'v': 2, 'key': [('name', 1)]}}


In [ ]:
# Menghapus database
# Hati-hati dengan operasi ini karena akan menghapus seluruh database!
client.drop_database('demoDB')

In [24]:
# Menggunakan agregasi untuk menghitung jumlah pengguna berdasarkan gender

for result in db.users.aggregate([
    {'$group': {'_id': '$gender', 'count': {'$sum': 1}}}
]):
    print(result)

{'_id': 'female', 'count': 2}
{'_id': 'male', 'count': 2}


In [26]:
# Mencari rata-rata umur pengguna

average_age = db.users.aggregate([
    {'$group': {'_id': None, 'averageAge': {'$avg': '$age'}}}
])
for avg in average_age:
    print(avg)

{'_id': None, 'averageAge': 27.75}


In [27]:
# Mengurutkan pengguna berdasarkan umur

for user in db.users.aggregate([{'$sort': {'age': -1}}]):
    print(user)


{'_id': ObjectId('654a1b25d61c91fe5de86633'), 'name': 'Bob', 'age': 32, 'address': '789 Maple St', 'gender': 'male'}
{'_id': ObjectId('654a1b25d61c91fe5de86635'), 'name': 'Diana', 'age': 29, 'address': '202 Oak St', 'gender': 'female'}
{'_id': ObjectId('654a1b25d61c91fe5de86632'), 'name': 'Alice', 'age': 28, 'address': '456 Elm St', 'gender': 'female'}
{'_id': ObjectId('654a1b25d61c91fe5de86634'), 'name': 'Charlie', 'age': 22, 'address': '101 Pine St', 'gender': 'male'}


In [28]:
# Penggunaan `$project` untuk mengubah struktur dokumen output

for user in db.users.aggregate([
    {'$project': {'name': 1, 'age': 1, 'isAdult': {'$cond': [{'$gte': ['$age', 18]}, True, False]}}}
]):
    print(user)

{'_id': ObjectId('654a1b25d61c91fe5de86632'), 'name': 'Alice', 'age': 28, 'isAdult': True}
{'_id': ObjectId('654a1b25d61c91fe5de86633'), 'name': 'Bob', 'age': 32, 'isAdult': True}
{'_id': ObjectId('654a1b25d61c91fe5de86634'), 'name': 'Charlie', 'age': 22, 'isAdult': True}
{'_id': ObjectId('654a1b25d61c91fe5de86635'), 'name': 'Diana', 'age': 29, 'isAdult': True}


In [29]:
# Menggunakan `$match` untuk menyaring data

for user in db.users.aggregate([
    {'$match': {'gender': 'female'}},
    {'$sort': {'age': 1}}
]):
    print(user)

{'_id': ObjectId('654a1b25d61c91fe5de86632'), 'name': 'Alice', 'age': 28, 'address': '456 Elm St', 'gender': 'female'}
{'_id': ObjectId('654a1b25d61c91fe5de86635'), 'name': 'Diana', 'age': 29, 'address': '202 Oak St', 'gender': 'female'}


In [30]:
# Penggunaan `$bucket` untuk membagi data ke dalam rentang

for bucket in db.users.aggregate([
    {
        '$bucket': {
            'groupBy': '$age',
            'boundaries': [20, 30, 40],
            'default': 'other',
            'output': {
                'count': {'$sum': 1},
                'users': {'$push': '$name'}
            }
        }
    }
]):
    print(bucket)

{'_id': 20, 'count': 3, 'users': ['Alice', 'Charlie', 'Diana']}
{'_id': 30, 'count': 1, 'users': ['Bob']}


In [31]:
# `$unwind`: Menguraikan array sehingga setiap elemennya menjadi dokumen tersendiri

db.users.insert_one({'name': 'Eve', 'hobbies': ['reading', 'hiking', 'swimming']})
for hobby in db.users.aggregate([{'$unwind': '$hobbies'}]):
    print(hobby)

{'_id': ObjectId('654a1bdbd61c91fe5de86636'), 'name': 'Eve', 'hobbies': 'reading'}
{'_id': ObjectId('654a1bdbd61c91fe5de86636'), 'name': 'Eve', 'hobbies': 'hiking'}
{'_id': ObjectId('654a1bdbd61c91fe5de86636'), 'name': 'Eve', 'hobbies': 'swimming'}


In [32]:
# `$lookup`: Memungkinkan Anda untuk melakukan join antar koleksi

db.orders.insert_many([
    {'userId': 'John', 'item': 'book', 'price': 20},
    {'userId': 'Alice', 'item': 'pen', 'price': 1}
])

for user in db.users.aggregate([
    {
        '$lookup': {
            'from': 'orders',
            'localField': 'name',
            'foreignField': 'userId',
            'as': 'userOrders'
        }
    }
]):
    print(user)


{'_id': ObjectId('654a1b25d61c91fe5de86632'), 'name': 'Alice', 'age': 28, 'address': '456 Elm St', 'gender': 'female', 'userOrders': [{'_id': ObjectId('654a1be3d61c91fe5de86638'), 'userId': 'Alice', 'item': 'pen', 'price': 1}]}
{'_id': ObjectId('654a1b25d61c91fe5de86633'), 'name': 'Bob', 'age': 32, 'address': '789 Maple St', 'gender': 'male', 'userOrders': []}
{'_id': ObjectId('654a1b25d61c91fe5de86634'), 'name': 'Charlie', 'age': 22, 'address': '101 Pine St', 'gender': 'male', 'userOrders': []}
{'_id': ObjectId('654a1b25d61c91fe5de86635'), 'name': 'Diana', 'age': 29, 'address': '202 Oak St', 'gender': 'female', 'userOrders': []}
{'_id': ObjectId('654a1bdbd61c91fe5de86636'), 'name': 'Eve', 'hobbies': ['reading', 'hiking', 'swimming'], 'userOrders': []}


In [33]:
# `$facet`: Memungkinkan Anda untuk membuat banyak jalur pemrosesan dalam satu tahapan agregasi

for facet in db.users.aggregate([
    {
        '$facet': {
            'byGender': [{'$group': {'_id': '$gender', 'count': {'$sum': 1}}}],
            'byAge': [{'$bucket': {'groupBy': '$age', 'boundaries': [20, 30, 40], 'default': 'other'}}]
        }
    }
]):
    print(facet)

{'byGender': [{'_id': 'female', 'count': 2}, {'_id': None, 'count': 1}, {'_id': 'male', 'count': 2}], 'byAge': [{'_id': 20, 'count': 3}, {'_id': 30, 'count': 1}, {'_id': 'other', 'count': 1}]}


In [34]:
# `$addFields` atau `$set`: Menambahkan bidang baru ke dokumen

for user in db.users.aggregate([{'$addFields': {'doubleAge': {'$multiply': ['$age', 2]}}}]):
    print(user)

{'_id': ObjectId('654a1b25d61c91fe5de86632'), 'name': 'Alice', 'age': 28, 'address': '456 Elm St', 'gender': 'female', 'doubleAge': 56}
{'_id': ObjectId('654a1b25d61c91fe5de86633'), 'name': 'Bob', 'age': 32, 'address': '789 Maple St', 'gender': 'male', 'doubleAge': 64}
{'_id': ObjectId('654a1b25d61c91fe5de86634'), 'name': 'Charlie', 'age': 22, 'address': '101 Pine St', 'gender': 'male', 'doubleAge': 44}
{'_id': ObjectId('654a1b25d61c91fe5de86635'), 'name': 'Diana', 'age': 29, 'address': '202 Oak St', 'gender': 'female', 'doubleAge': 58}
{'_id': ObjectId('654a1bdbd61c91fe5de86636'), 'name': 'Eve', 'hobbies': ['reading', 'hiking', 'swimming'], 'doubleAge': None}
